<a href="https://colab.research.google.com/github/Dr-Blank-alt/BP/blob/main/Post_Preparing_template_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <h1><b>←</b> ឵឵ <font size="5">Mount Google Drive + Install Requirements</font>

#@markdown <h1><b></b> ឵឵ <font size="4">Extra options below to run script more faster </font>

#@markdown <ul><li>Do you want <b>Screenshots</b> to be Generated?</li>
generate_SS = False #@param {type:"boolean"}

#@markdown <ul><li>Will you be using <b>Imgur?</b></li>
use_imgur = False #@param {type:"boolean"}

#@markdown <ul><li>Will you be using <b>Catbox and/or uguu</b>?</li></ol>
use_catbox_uguu = False #@param {type:"boolean"}


from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

from google.colab import drive
drive.mount('/content/drive')
print('Drive is mounted. Installing dependencies.')

loadingAn(name="lds")
# Installing Essential Libraries
textAn("Installing Dependencies...", ty='twg')
os.system('apt install mediainfo')
os.system("pip install git+https://github.com/alberanid/imdbpy") #to get imdb id
os.system("pip install guessit")
os.system("pip install xmltodict")

if generate_SS: 
    os.system('apt-get install ffmpeg')
    if use_imgur: 
        !pip install pyimgur &>/dev/null
    
    if use_catbox_uguu:
        os.system("pip install pyupload")


clear_output()
print('Installation finished. Fill the details and generate the post!.')
clear_output(wait=True)


################### Importing Dependencies ##########################
import functools
import json
import os
import pathlib
import random
import re
import subprocess
import sys
import time
import urllib.request
from base64 import b64encode

import requests
import xmltodict
from bs4 import BeautifulSoup
from guessit import guessit
from imdb import IMDb
from IPython.display import clear_output

if generate_SS:
    if use_imgur:
        import pyimgur
    if use_catbox_uguu:
        import pyupload


######################## Custom Functions ################################

def path_leaf(path: str) -> str:
    """Returns file name or folder name only no matter the path"""
    head, tail = os.path.split(path)
    return tail or os.path.basename(head)

def getFileSize(file_path: str) -> str:
    """Returns the file Size in String Format as 'XX GB' or 'XX MB'

    Args:
        file_path (str): Path of the  file to get size
    """
    try:
        size_in_bytes = os.path.getsize(file_path)
    except:
        return ""

    return format_size(size_in_bytes)

def format_size(size_in_bytes: float) -> str:
  size_in_KB = size_in_bytes / pow(2, 10)
  size_in_MB = size_in_bytes / pow(2, 20)
  size_in_GB = size_in_bytes / pow(2, 30)

  if size_in_GB < 1:
    if size_in_MB < 1:
        return str(round(size_in_KB, 2)) + " KB"
    else:
        return str(round(size_in_MB, 2)) + " MB"

  else:
      return str(round(size_in_GB, 2)) + " GB"


def get_size_path(path: str) -> str:
  if os.path.isfile(path):
    return getFileSize(path)
  else:
    org_path = os.getcwd()
    os.chdir(path)
    size_in_bytes = sum(os.path.getsize(x) for x in os.listdir(path))
    os.chdir(org_path)
    return format_size(size_in_bytes)

def get_size_media(path: str):
  """Either gets size of the movie or the whole folder in which the episode 
      is placed.  
  """
  file_name = os.path.basename(path)

  dict = guessit(file_name)
  if dict['type'] == 'episode':
    return get_size_path(os.path.split(path)[0])
  else:
    return get_size_path(path)

def getTrailerIMDbApi(imdb_id: str, imdb_api_key: str) -> str:
  """Returns the YT Url of the given imdb title. Returns empty string if not found"""
  imdb_api_url = f"https://imdb-api.com/API/YouTubeTrailer/{imdb_api_key}/{imdb_id}"
  with urllib.request.urlopen(imdb_api_url) as response:
    YT_data = json.loads(response.read().decode())
  if not YT_data['errorMessage']:
    return YT_data['videoUrl'] 
  return ""


def uploadIMG(image_path: str, host: str = "imgur", try_other: bool = True) -> str:
    """returns url after upload the image to the given host; depends on py upload

    Args:
        image_path (str): the path to the image to upload
        host (str, optional): the image is uploaded to this host. 
                                currently supports five ["catbox", "mixtape", "uguu", "fileio", "imgur"]. 
                                Defaults to "imgur".
        try_other (bool): If true, tries other uploaders to get an url for the image
    Returns:
        str: URL of the image uploaded
    """

    available_hosts=["catbox", "uguu", "mixtape"]
    if host == "imgur":
        if not imgur_api_client_id:
            raise ValueError("Imgur Api Client ID Missing \n Get the Id from: https://api.imgur.com/oauth2/addclient")
        imgur_API = pyimgur.Imgur(imgur_api_client_id)
        image_url = imgur_API.upload_image(image_path).link
        if image_url:
            return image_url
        else:
            host = "catbox"


    result = subprocess.run(
        ["pyupload", "--host", host, image_path], stdout=subprocess.PIPE,
    )

    if not result.returncode:  # returncode 0 = success
        pattern = r"Your link : (.*)\n"
        image_url = re.findall(pattern, result.stdout.decode("utf-8"))
        return image_url[0]
    elif try_other:
        available_hosts.remove(host)
        for other_host in available_hosts:
            image_url = uploadIMG(
                image_path=image_path, host=other_host, try_other=False
            )
            if image_url:
                return image_url
        return ""
    else:
        return ""

def get_media_info_str(Path: str) -> str:
    try:
        os.remove("/root/.nfo")
    except:
        pass
    subprocess.call(
        ["mediainfo", "--logfile=/root/.nfo", Path], stdout=open(os.devnull, "wb")
    )
    nfo = open("/root/.nfo", "r", encoding="utf-8").readlines()
    nfo.pop()
    del nfo[1]
    nfo[1] = "Complete Name                            : {}\n".format(
        os.path.basename(Path)
    )
    try:
        os.remove("/root/.nfo")
    except:
        pass
    return "".join(nfo)


def get_media_info_dict(Path: str) -> object:
    try:
        os.remove("/root/.xml")
    except:
        pass
    subprocess.call(
        ["mediainfo", "--logfile=/root/.xml", "--Output=XML", Path],
        stdout=open(os.devnull, "wb"),
    )
    nfo = open("/root/.xml", "r", encoding="utf-8").read()
    try:
        os.remove("/root/.xml")
    except:
        pass
    return xmltodict.parse(nfo)



def generate_title_mediainfo(Path: str, mediaInfo: dict, pass_thru: bool = True) -> str:
    """returns a title string according to rules of BP including all the required info

    Args:
        Path (str): The path of the media file from which title is to be generated [video file]
        mediaInfo (dict): orderedDict object retruned from function "get_media_info_dict(Path)"
        pass_thru (bool, optional): If True, function returns title ignoring mediaInfo by just appending size to file name. Defaults to True.

    Returns:
        str: Title string including the name, year, season(if req), video height, bit depth, audio channels, audio codec, video codec and scene group
    """

    file_name = path_leaf(Path)
    guessit_dict = guessit(file_name)

    # brings the path back to one level down which was moved by "path_leaf"
    if guessit_dict["type"] == "episode":
        file_name = os.path.basename(os.path.split(Path)[0]) + os.path.splitext(Path)[1]

    if pass_thru:
        return os.path.splitext(file_name)[0] + " | " + get_size_media(Path)

    # list of unique languages in audio tracks
    unique_languages = list(
        set(
            track.get(
                "Language", imdb_data.get("Language", "English").split(", ")[0].lower()
            )
            for track in mediaInfo["MediaInfo"]["media"]["track"]
            if track["@type"] == "Audio"
        )
    )

    # list of all video tracks
    video_tracks = [
        track
        for track in mediaInfo["MediaInfo"]["media"]["track"]
        if track["@type"] == "Video"
    ]

    # list of all audio tracks
    audio_tracks = [
        track
        for track in mediaInfo["MediaInfo"]["media"]["track"]
        if track["@type"] == "Audio"
    ]

    # Movie/Series Name + (Year)
    title_dump = f"{imdb_data['Title']} ({imdb_data['Year'].split('–')[0]}) "

    # Adding S00 if series
    title_dump += (
        f"S{str(guessit_dict.get('season', '')).zfill(2)} "
        if guessit_dict["type"] == "episode"
        else ""
    )

    # Language: Multi if more than one language, else just the language in MediInfo
    title_dump += "multi " if len(unique_languages) > 1 else f"({unique_languages[0]}) "

    # screen size: 720p/1080p/2160p
    title_dump += (
        f"({guessit_dict.get('screen_size', '{}p'.format(video_tracks[0]['Height']))} "
    )

    # Bit Depth: 8bit/10bit/12bit
    title_dump += (
        f"{video_tracks[0]['BitDepth']}bit "
        if video_tracks[0].get("BitDepth", "") in [10, 12]
        else f"{guessit_dict['color_depth'].replace('-', '')} "
        if guessit_dict.get("color_depth", "") in ["10-bit", "12-bit"]
        else ""
    )

    # source: web/Bluray/DvD
    title_dump += f"{guessit_dict.get('source', '???')} "

    # other: REMUX/PROPER/Dual Audio/Rip/ReEncoded
    title_dump += (
        (" ".join(guessit_dict["other"]) + " ")
        if isinstance(guessit_dict.get("other", ""), list)
        else (guessit_dict["other"] + " ")
        if guessit_dict.get("other", "")
        else ""
    )

    # Audio Info: 2CH/6CH/8CH AAC/DDP/OPUS
    title_dump += (
        f"{audio_tracks[0]['Channels'].split(' / ')[0]}CH {audio_tracks[0]['Format']} "
    )

    # Video Info: HEVC/AVC/Xvid
    title_dump += f"{video_tracks[0]['Format']}) "

    # Release Group: Telly/Streamion/PSA/QxR
    title_dump += (
        f"[{guessit_dict['release_group']}] "
        if guessit_dict.get("release_group", "")
        else ""
    )
    title_dump += f"| {get_size_media(Path)}"

    return title_dump


#################### END of Functions #############################

####################### Templates #######################

def DrB_OG_Template(
    text_to_hide: str,
    imdb_data: dict = None,
    YouTube_URL: str = "",
    source_string: str = "",
    media_info: str = "",
    SSURLs: list = None,
    min_posts: int = 0,
    min_reaction_score: int = 0,
) -> str:
    if not imdb_data:
        raise ValueError(
            "imdb_data not provided: This the json response recieved from OMDbAPI.com"
        )
    imdb_id = imdb_data["imdbID"]
    imdbURL = "https://www.imdb.com/title/{}".format(imdb_id)
    imdbRating = (
        "[SIZE=8][B]{}[/B]/10[/SIZE]".format(imdb_data.get("imdbRating"))
        if imdb_data.get("imdbRating", "")
        else ""
    )
    imdbVotes = (
        "[SIZE=6][IMG]https://i.imgur.com/sEpKj3O.png[/IMG]{}[/SIZE]\n".format(
            imdb_data.get("imdbVotes")
        )
        if imdb_data.get("imdbVotes", "")
        else ""
    )

    bpIMDbSearchUrl = f"https://blackpearl.biz/search/1/?q={imdb_id}&o=date"
    HQ_poster = re.sub("[._V1_SX300]{10}", "", str(imdb_data["Poster"]))

    # Title
    text_dump = "[FORUMCOLOR][CENTER][SIZE=7][URL={}]{} ({})[/URL][/FORUMCOLOR][/SIZE]".format(
        bpIMDbSearchUrl, imdb_data["Title"], imdb_data["Year"]
    )
    # Poster
    text_dump += "[IMG width='450px']{}[/IMG][/CENTER]\n".format(HQ_poster)

    # IMDb
    text_dump += "[CENTER][URL={}][IMG]https://i.imgur.com/rcSipDw.png[/IMG][/URL] {}\n{}[/CENTER][HR][/HR]".format(
        imdbURL, imdbRating, imdbVotes
    )
    # Plot
    if imdb_data.get("Plot", ""):
        text_dump += "[FORUMCOLOR][FONT=Montserrat][SIZE=6][B]Plot[/B][/SIZE][/FONT][/FORUMCOLOR]\n"
        text_dump += "[INDENT]{}[/INDENT][HR][/HR]".format(imdb_data["Plot"])
    # YT
    if YouTube_URL:
        text_dump += "[FORUMCOLOR][FONT=Montserrat][SIZE=6][B]Trailer[/B][/SIZE][/FONT][/FORUMCOLOR]\n"
        text_dump += "[CENTER][MEDIA=youtube]{}[/MEDIA][/CENTER]\n".format(
            re.search("([A-Za-z0-9-_]){11}", YouTube_URL).group(0)
        )
        text_dump += "[HR][/HR]"

    # MoreInfo
    text_dump += "[FORUMCOLOR][FONT=Montserrat][SIZE=6][B]Movie Info[/B][/SIZE][/FONT][/FORUMCOLOR][LIST]"

    text_dump += (
        f"[*][FORUMCOLOR]Rating:[/FORUMCOLOR] {imdb_data['Rated']}\n"
        if "Rated" in imdb_data.keys() and not imdb_data["Rated"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Genre:[/FORUMCOLOR] {imdb_data['Genre']}\n"
        if "Genre" in imdb_data.keys() and not imdb_data["Genre"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Directed By:[/FORUMCOLOR] {imdb_data['Director']}\n"
        if "Director" in imdb_data.keys() and not imdb_data["Director"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Written By:[/FORUMCOLOR] {imdb_data['Writer']}\n"
        if "Writer" in imdb_data.keys() and not imdb_data["Writer"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Starring:[/FORUMCOLOR] {imdb_data['Actors']}\n"
        if "Actors" in imdb_data.keys() and not imdb_data["Actors"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Release Date:[/FORUMCOLOR] {imdb_data['Released']}\n"
        if "Released" in imdb_data.keys() and not imdb_data["Released"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Runtime:[/FORUMCOLOR] {imdb_data['Runtime']}\n"
        if "Runtime" in imdb_data.keys() and not imdb_data["Runtime"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Production:[/FORUMCOLOR] {imdb_data['Production']}\n"
        if "Production" in imdb_data.keys() and not imdb_data["Production"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Awards:[/FORUMCOLOR] {imdb_data['Awards']}\n"
        if "Awards" in imdb_data.keys() and not imdb_data["Awards"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR]Language:[/FORUMCOLOR] {imdb_data['Language']}\n"
        if "Language" in imdb_data.keys() and not imdb_data["Language"] == "N/A"
        else ""
    )

    text_dump += "[/LIST][HR][/HR]"

    # Source
    if source_string:
        text_dump += f"[FORUMCOLOR][FONT=Montserrat][SIZE=6][B]Source[/B][/SIZE][/FONT][/FORUMCOLOR]\n[INDENT]{source_string}[/INDENT][HR][/HR]"

    # MEDIAINFO
    if media_info:
        text_dump += "[FORUMCOLOR]"
        text_dump += "[FONT=Montserrat][SIZE=6][B]MediaInfo[/B][/SIZE][/FONT]"
        text_dump += "[/FORUMCOLOR][INDENT][SPOILER=Click to view MediaInfo]\n"
        text_dump += "[CODE]"
        text_dump += "{}".format(media_info)
        text_dump += "[/CODE]"
        text_dump += "[/SPOILER][/INDENT][HR][/HR]"
    # SS
    if SSURLs:
        text_dump += (
            "[FORUMCOLOR][FONT=Montserrat][SIZE=6][B]Screenshots[/B][/SIZE][/FONT]"
        )
        text_dump += "[/FORUMCOLOR][INDENT][SPOILER=Click to view Screenshots]\n"
        text_dump += "[CENTER]\n"
        text_dump += "".join(
            "[IMG width='800px']{}[/IMG]\n".format(url) for url in SSURLs
        )
        text_dump += "[/CENTER][/SPOILER][/INDENT][HR][/HR]\n\n"

    # Download Link
    text_dump += "[CENTER][FORUMCOLOR][SIZE=6][FONT=Trebuchet MS]Download Link[/FONT][/SIZE][/FORUMCOLOR]\n"
    if min_posts:
        text_dump += "[HIDEPOSTS={}]".format(min_posts)
    if min_reaction_score:
        text_dump += "[HIDEREACTSCORE={}]".format(min_reaction_score)

    text_dump += "[HIDEREACT=1,2,3,4,5,6,7,8,9]{}[/HIDEREACT][/CENTER]".format(
        text_to_hide
    )

    if min_posts:
        text_dump += "[/HIDEPOSTS]"
    if min_reaction_score:
        text_dump += "[/HIDEREACTSCORE]"

    return text_dump


def DrB_Minimal_Template(
    text_to_hide: str,
    imdb_data: dict = None,
    YouTube_URL: str = "",
    source_string: str = "",
    media_info: str = "",
    SSURLs: list = None,
    min_posts: int = 0,
    min_reaction_score: int = 0,
) -> str:
    if not imdb_data:
        raise ValueError(
            "imdb_data not provided: This the json response recieved from OMDbAPI.com"
        )
    imdb_id = imdb_data["imdbID"]
    imdbURL = "https://www.imdb.com/title/{}".format(imdb_id)
    imdbRating = (
        "[SIZE=8][B]{}[/B]/10[/SIZE]".format(imdb_data.get("imdbRating"))
        if imdb_data.get("imdbRating", "")
        else ""
    )
    imdbVotes = (
        "[SIZE=6][IMG]https://i.imgur.com/sEpKj3O.png[/IMG]{}[/SIZE]\n".format(
            imdb_data.get("imdbVotes")
        )
        if imdb_data.get("imdbVotes", "")
        else ""
    )

    bpIMDbSearchUrl = f"https://blackpearl.biz/search/1/?q={imdb_id}&o=date"
    # LQ_poster = "{}".format(imdb_data['Poster'])
    HQ_poster = re.sub("[._V1_SX300]{10}", "", str(imdb_data["Poster"]))

    # Title
    text_dump = "[FORUMCOLOR][CENTER][SIZE=7][URL={}]{} ({})[/URL][/FORUMCOLOR][/SIZE]".format(
        bpIMDbSearchUrl, imdb_data["Title"], imdb_data["Year"]
    )

    # Poster
    text_dump += "[IMG width='450px']{}[/IMG][/CENTER]\n".format(HQ_poster)

    # IMDb
    text_dump += "[CENTER][URL={}][IMG]https://i.imgur.com/rcSipDw.png[/IMG][/URL] {}\n{}[/CENTER][HR][/HR]".format(
        imdbURL, imdbRating, imdbVotes
    )
    # Plot
    text_dump += "[TABS][SLIDE_HEADER][FONT=Montserrat][SIZE=6]Plot[/SIZE][/FONT][/SLIDE_HEADER]\n"
    text_dump += "[SLIDE][CENTER]{}[/B][/CENTER][/SLIDE]".format(imdb_data["Plot"])

    # YT
    if YouTube_URL:
        text_dump += "[SLIDE_HEADER][FONT=Montserrat][SIZE=6]Trailer[/SIZE][/FONT]\n[/SLIDE_HEADER][SLIDE]"
        text_dump += "[CENTER][MEDIA=youtube]{}[/MEDIA][/CENTER]\n[/SLIDE]".format(
            re.search("([A-Za-z0-9-_]){11}", YouTube_URL).group(0)
        )

    # MoreInfo
    text_dump += "[SLIDE_HEADER][FONT=Montserrat][SIZE=6]Info[/SIZE][/FONT][/SLIDE_HEADER][SLIDE][LIST]"

    text_dump += (
        f"[*][FORUMCOLOR][B]Rating:[/B][/FORUMCOLOR] {imdb_data['Rated']}\n"
        if "Rated" in imdb_data.keys() and not imdb_data["Rated"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Genre:[/B][/FORUMCOLOR] {imdb_data['Genre']}\n"
        if "Genre" in imdb_data.keys() and not imdb_data["Genre"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Directed By:[/B][/FORUMCOLOR] {imdb_data['Director']}\n"
        if "Director" in imdb_data.keys() and not imdb_data["Director"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Written By:[/B][/FORUMCOLOR] {imdb_data['Writer']}\n"
        if "Writer" in imdb_data.keys() and not imdb_data["Writer"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Starring:[/B][/FORUMCOLOR] {imdb_data['Actors']}\n"
        if "Actors" in imdb_data.keys() and not imdb_data["Actors"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Release Date:[/B][/FORUMCOLOR] {imdb_data['Released']}\n"
        if "Released" in imdb_data.keys() and not imdb_data["Released"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Runtime:[/B][/FORUMCOLOR] {imdb_data['Runtime']}\n"
        if "Runtime" in imdb_data.keys() and not imdb_data["Runtime"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Production:[/B][/FORUMCOLOR] {imdb_data['Production']}\n"
        if "Production" in imdb_data.keys() and not imdb_data["Production"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Awards:[/B][/FORUMCOLOR] {imdb_data['Awards']}\n"
        if "Awards" in imdb_data.keys() and not imdb_data["Awards"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][FORUMCOLOR][B]Language:[/B][/FORUMCOLOR] {imdb_data['Language']}\n"
        if "Language" in imdb_data.keys() and not imdb_data["Language"] == "N/A"
        else ""
    )

    text_dump += "[/LIST][/SLIDE]"

    # Source
    if source_string:
        text_dump += f"[SLIDE_HEADER][FONT=Montserrat][SIZE=6]Source[/SIZE][/FONT][/SLIDE_HEADER][SLIDE][CENTER]{source_string}[/CENTER][/SLIDE]"

    # MEDIAINFO
    if media_info:
        text_dump += "[SLIDE_HEADER][FONT=Montserrat][SIZE=6]MediaInfo[/SIZE][/FONT][/SLIDE_HEADER][SLIDE][CENTER]"
        text_dump += "[CODE]{}[/CODE][/CENTER][/SLIDE]".format(media_info)

    # SS
    if SSURLs:
        text_dump += "[SLIDE_HEADER][FONT=Montserrat][SIZE=6]Screenshots[/SIZE][/FONT][/SLIDE_HEADER][SLIDE][CENTER]\n"
        text_dump += "".join(
            "[IMG width='800px']{}[/IMG]\n".format(url) for url in SSURLs
        )
        text_dump += "[/CENTER]\n\n"
        text_dump += "[/SLIDE]"

    text_dump += "[/TABS]"

    # Download Link
    text_dump += "[HR][/HR][CENTER][FORUMCOLOR][SIZE=6][FONT=Trebuchet MS]Download Link[/FONT][/SIZE][/FORUMCOLOR]\n"
    if min_posts:
        text_dump += "[HIDEPOSTS={}]".format(min_posts)
    if min_reaction_score:
        text_dump += "[HIDEREACTSCORE={}]".format(min_reaction_score)

    text_dump += "[HIDEREACT=1,2,3,4,5,6,7,8,9]{}[/HIDEREACT][/CENTER]".format(
        text_to_hide
    )

    if min_posts:
        text_dump += "[/HIDEPOSTS]"
    if min_reaction_score:
        text_dump += "[/HIDEREACTSCORE]"

    return text_dump


def BP_UserScript_Template(
    text_to_hide: str,
    imdb_data: dict = None,
    YouTube_URL: str = "",
    source_string: str = "",
    media_info: str = "",
    SSURLs: list = None,
    min_posts: int = 0,
    min_reaction_score: int = 0,
) -> str:
    if not imdb_data:
        raise ValueError(
            "imdb_data not provided: This the json response recieved from OMDbAPI.com"
        )
    imdb_id = imdb_data["imdbID"]
    imdbURL = "https://www.imdb.com/title/{}".format(imdb_id)
    imdbRating = (
        "[SIZE=6][B]{}[/B]/10[/SIZE]".format(imdb_data.get("imdbRating"))
        if imdb_data.get("imdbRating", "")
        else ""
    )
    imdbVotes = (
        "[SIZE=6][IMG]https://i.imgur.com/sEpKj3O.png[/IMG]{}[/SIZE]\n".format(
            imdb_data.get("imdbVotes")
        )
        if imdb_data.get("imdbVotes", "")
        else ""
    )

    bpIMDbSearchUrl = f"https://blackpearl.biz/search/1/?q={imdb_id}&o=date"
    LQ_poster = "{}".format(imdb_data["Poster"])

    # Poster
    text_dump = "[CENTER][IMG]{}[/IMG]\n".format(LQ_poster)

    # Title
    text_dump += "[FORUMCOLOR][B][SIZE=6][URL={}]{} ({})[/URL][/FORUMCOLOR][/SIZE][/B][/CENTER]".format(
        bpIMDbSearchUrl, imdb_data["Title"], imdb_data["Year"]
    )
    # IMDb
    text_dump += "[CENTER][URL={}][IMG]https://i.imgur.com/rcSipDw.png[/IMG][/URL] {}\n{}[/CENTER][HR][/HR]".format(
        imdbURL, imdbRating, imdbVotes
    )
    # Plot
    if imdb_data.get("Plot", ""):
        text_dump += (
            "[INDENT][FORUMCOLOR][B][SIZE=6]Plot[/SIZE][/B][/FORUMCOLOR][/INDENT]\n\n"
        )
        text_dump += "{}\n[HR][/HR]".format(imdb_data["Plot"])
    # YT
    if YouTube_URL:
        text_dump += (
            "[INDENT][FORUMCOLOR][B][SIZE=6]Trailer[/SIZE][/B][/FORUMCOLOR][/INDENT]\n"
        )
        text_dump += "[MEDIA=youtube]{}[/MEDIA]\n".format(
            re.search("([A-Za-z0-9-_]){11}", YouTube_URL).group(0)
        )
        text_dump += "[HR][/HR]"

    # SS
    if SSURLs:
        text_dump += "[INDENT][FORUMCOLOR][B][SIZE=6]Screenshots[/SIZE][/B]"
        text_dump += "[/FORUMCOLOR][/INDENT][SPOILER='Click here to view Screenshots']\n[CENTER]\n"
        text_dump += "".join("[IMG]{}[/IMG]\n".format(url) for url in SSURLs)
        text_dump += "[/CENTER][/SPOILER]\n[HR][/HR]\n"

    # MoreInfo
    text_dump += "[INDENT][FORUMCOLOR][B][SIZE=6]Movie Info[/SIZE][/B][/FORUMCOLOR][/INDENT][LIST]"

    text_dump += (
        f"[*][B]Rating:[/B] {imdb_data['Rated']}\n"
        if "Rated" in imdb_data.keys() and not imdb_data["Rated"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Genre:[/B] {imdb_data['Genre']}\n"
        if "Genre" in imdb_data.keys() and not imdb_data["Genre"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Directed By:[/B] {imdb_data['Director']}\n"
        if "Director" in imdb_data.keys() and not imdb_data["Director"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Written By:[/B] {imdb_data['Writer']}\n"
        if "Writer" in imdb_data.keys() and not imdb_data["Writer"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Starring:[/B] {imdb_data['Actors']}\n"
        if "Actors" in imdb_data.keys() and not imdb_data["Actors"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Release Date:[/B] {imdb_data['Released']}\n"
        if "Released" in imdb_data.keys() and not imdb_data["Released"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Runtime:[/B] {imdb_data['Runtime']}\n"
        if "Runtime" in imdb_data.keys() and not imdb_data["Runtime"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Production:[/B] {imdb_data['Production']}\n"
        if "Production" in imdb_data.keys() and not imdb_data["Production"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Awards:[/B] {imdb_data['Awards']}\n"
        if "Awards" in imdb_data.keys() and not imdb_data["Awards"] == "N/A"
        else ""
    )
    text_dump += (
        f"[*][B]Language:[/B] {imdb_data['Language']}\n"
        if "Language" in imdb_data.keys() and not imdb_data["Language"] == "N/A"
        else ""
    )

    text_dump += "[/LIST][HR][/HR]"

    # Source
    if source_string:
        text_dump += f"[INDENT][FORUMCOLOR][B][SIZE=6]Source[/SIZE][/B][/FORUMCOLOR][/INDENT]\n\n{source_string}[HR][/HR]"

    # MEDIAINFO
    if media_info:
        text_dump += "[INDENT][FORUMCOLOR][B][SIZE=6]Media Info [/SIZE][/B][/FORUMCOLOR][/INDENT][SPOILER=Click here to view Media Info]\n"
        text_dump += "[CODE]{}[/CODE][/SPOILER][HR][/HR]".format(media_info)

    # Download Link
    text_dump += (
        "[CENTER][FORUMCOLOR][SIZE=6][B]Download Link[/B][/SIZE][/FORUMCOLOR]\n"
    )
    if min_posts:
        text_dump += "[HIDEPOSTS={}]".format(min_posts)
    if min_reaction_score:
        text_dump += "[HIDEREACTSCORE={}]".format(min_reaction_score)

    text_dump += "[HIDEREACT=1,2,3,4,5,6,7,8,9]{}[/HIDEREACT][/CENTER]".format(
        text_to_hide
    )

    if min_posts:
        text_dump += "[/HIDEPOSTS]"
    if min_reaction_score:
        text_dump += "[/HIDEREACTSCORE]"

    return text_dump



## <img src='https://i.imgur.com/N4bdPff.png' height="25" alt="IMDB"/> **Generate BP template**
<ol>
<li><font size="3"><i>Generate your own Free OMDb API Key here: http://www.omdbapi.com/apikey.aspx</font>
<li><font size="3"><i>Generate your own Free imdb-api Key (OPTIONAL) after registering here: https://imdb-api.com/Identity/Account/Register</font>
<li><font size="3"><i>Generate your own Free Imgur Api Client id (OPTIONAL) after registering here: https://api.imgur.com/oauth2/addclient</font>


In [ ]:
Auto_UP_Gdrive = False  
AUTO_MOVE_PATH = "/content" 
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    findProcess,
    loadingAn,
)

%cd "/content"
%rm -rf "/content/frames"
#@markdown <h6><center><img src="https://i.imgur.com/FaxnTtv.png" width="900px"/></center>
#@markdown <center><font size="6" color="#56b84c">⸻⸻ <img src='https://yuju.pw/y/pcJm.png' height="70"> ⸻⸻</center>
#@markdown <center></font></center>

################### Variables ####################

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:21.0) Gecko/20100101 Firefox/21.0'}
#@markdown <h1><font size="6">» Fill </font><font size="6">API Keys:</font></center>
omdb_api_key = '' #@param {type:"string"}
imdb_api_key = '' #@param {type:"string"}
imgur_api_client_id = '' #@param {type:"string"}
#@markdown <font size="2" color=pink> * "imdb_api_key" is optional, only if you want find YT Trailer **accurately** [Reduces Speed though]. </font>

#@markdown <font size="2" color=pink> * "imgur_api_client_id" is optional but needed if you want to upload to Imgur. </font>

#@markdown <h1><font size="6">» Select Template</font></center>
template_to_use = 'DrB_Minimal_Template' #@param ["DrB_OG_Template", "DrB_Minimal_Template", "BP_UserScript_Template"]

#@markdown <font size="6" color="#E9A3C4">» Movie </font><font color="#9FA5F3"><font size="5">឵឵OR឵</font><font size="6" color="#E9A3C4"> TV File Info:</font>
Path = "/content/drive/path/to/movie/or/episode/or/series/folder" #@param {type:"string"}
Link = "https://drive.google.com/drive/folders/1CEUvxWmb-AtUESR93UAqeFRIxNok?usp=sharing" #@param {type:"string"}

source_string = "" #@param {type:"string"}
#@markdown <font size="2" color=pink>*Only for Encoders. Leave "source_string" blank if you are not posting your own encodes</font>

#@markdown <h1><font size="5">» ScreenShot Settings</font></center>
Host = 'imgur' #@param ["imgur", "catbox", "uguu", "mixtape"]
number_of_SS = 0 #@param {type:"integer"}
#@markdown <font size="3">Choose the host you want to upload your ScreenShots to and how many.[Limit: 5]</font>
#-ve number of SS will be 0 and more than 5 will be 5.
if number_of_SS < 0:
    number_of_SS = 0
elif number_of_SS > 5:
    number_of_SS = 5
    
#@markdown <h1><font size="5">» Hide Link Settings</font></center>
min_reaction_score = 5 #@param {type:"integer"}
min_posts = 3 #@param {type:"integer"}
#@markdown <font size="2" color=pink> Choose the minimum number of reactions the leecher should have and the minimum number of posts he/she should have before leeching your link.</font>

#@markdown <h1><font size="6" color="#F1F290">» IMDb</font><font size="6"> URL <small>or</small> ID:</font></center>
IMDB_URL_OR_ID = '' #@param {type:"string"}
#@markdown <font size="2" color=pink> * Only fill IMDb if the result was not what you expected. Leave it blank most of the time as it will work.</font>

#@markdown <h1><font size="6" color="#F6675E">» YouTube</font><font size="6"> URL:</center>
YouTube_URL = '' #@param {type:"string"}
#@markdown <font size="2" color=pink> * Only fill Youtube URL if the result was not what you expected. Leave it blank most of the time as it will work on its own</font>

#@markdown <h1><font size="5">» Extra Settings</font></center>
generate_title = True #@param {type:"boolean"}
pass_thru = False #@param {type:"boolean"}
#@markdown <font size="2" color=pink> * when you turn **on** pass_thru, the title string will be the filename **as it is** followed by the **size** of movie or series folder</font>

#@markdown <font size="2" color=pink> * when you turn **off** pass_thru, the title string will be reconstructed from scratch using **mediaInfo**</font>

use_base64 = False #@param {type:"boolean"}


###################### Some Checks #####################
if not os.path.exists(Path):
    clear_output()
    raise ValueError("Your video filepath does not exist, please check it.")

if not os.path.isfile(Path):
    flag = False
    for file in os.listdir(Path):
        if os.path.splitext(file)[1] in [".mkv", ".mp4"]:
            Path = os.path.join(Path, file)
            flag = True
            break
    if not flag:
        sys.exit("Your Path is a Folder. Please enter a video file path.")

if not omdb_api_key:
    clear_output()
    sys.exit("Please enter the \"omdb_api_key\"")

if not Link:
    sys.exit("You forgot to put in the link to share!")

if not generate_SS:
    number_of_SS = 0

if Host in ["uguu", "catbox"] and not use_catbox_uguu:
    raise NameError("'use_catbox_uguu' is disabled in the first cell and you have selected the host as 'uguu' or 'catbox'.\n Enable the checkbox in front of 'use_catbox_uguu' and run cell 1 again and then continue OR select another host.")

if Host == "imgur" and not use_imgur:
    raise NameError("'use_imgur' is disabled in the first cell and you have selected the host as 'imgur'.\n Enable the checkbox in front of 'use_imgur' and run cell 1 again and then continue OR select another host.")


#############################################

# IMDb id guessing #
my_file = path_leaf(Path)
dict = guessit(my_file)
movie_year = dict.get('year',"")
parsed_name = dict['title']  if not movie_year else dict['title'] + ' (' + str(movie_year) + ')'

if not IMDB_URL_OR_ID:
    print("\nGuessing IMDb ID...")
    ia = IMDb()
    movies = ia.search_movie(parsed_name)
    try:
        IMDB_URL_OR_ID = "tt" + movies[0].movieID
    except IndexError as e:
        raise Exception("Could not find IMDb id >_< please paste the imdb id manually")

imdb_pattern = re.compile('(tt\d{7,8})')
imdb_id = re.findall(imdb_pattern, IMDB_URL_OR_ID)[0]

# Fetch data #
imdb_url = f'http://www.omdbapi.com/?i={imdb_id}&apikey={omdb_api_key}&r=json&plot=full'
with urllib.request.urlopen(imdb_url) as imdb_url:
    imdb_data = json.loads(imdb_url.read().decode())

# YT trailer finding if imdb_api exists #
print("\nFinding YouTube Trailer...")
if imdb_api_key:
    if not YouTube_URL:
        try:
            YouTube_URL = getTrailerIMDbApi(imdb_id, imdb_api_key)
        except:
            YouTube_URL = ""

# Try finding Trailer using requests if not in Db #
if not YouTube_URL:
    params={"search_query": parsed_name +"Official Trailer"}
    YT_LINK = urllib.request.urlopen("https://www.youtube.com/results?" + urllib.parse.urlencode(params))
    YouTube_URL = re.findall(r"watch\?v=(\S{11})",YT_LINK.read().decode())[0]



#################### Getting MediaInfo ####################
print("\nFetching MediaInfo...")

media_info = get_media_info_str(Path)
###################### Getting SS ########################
!mkdir -p "/content/frames"

SSURLs = []  
if number_of_SS:
    print("\nFetching Screenshots...")
    movie_folder_path, file_name = os.path.split(Path)
    os.chdir(movie_folder_path)

    output = subprocess.Popen(["ffmpeg", "-i", file_name], stderr=subprocess.PIPE).communicate()
    re_duration = re.compile(r"Duration: (.*?)\.")
    duration = re_duration.search(output[1].decode('utf-8')).groups()[0]
    seconds = functools.reduce(lambda x,y:x*60+y,map(int,duration.split(":")))
    i=1
    for x in range(200, seconds)[::int(seconds/number_of_SS)]:
        timexstamp = ":".join([x.zfill(2) for x in map(str,[x//3600,(x//60)%60,x%60])])+".0"
        output = subprocess.Popen(["ffmpeg", "-hide_banner", "-ss", timexstamp, "-i", file_name, "-vframes", "1", "-q:v", "2", "-y",  f'/content/frames/Preview-{i}.png'], stderr=subprocess.PIPE).communicate()
        i+=1

    # Uploading SS
    os.chdir("/content/frames")
    print("\nUploading Screenshots...")
    SSURLs = [uploadIMG(ss, host=Host) for ss in os.listdir()]


# DLink preparation #
if use_base64:
    encoded_url = b64encode(Link.encode("utf-8")).decode("utf-8")
    text_to_hide = f"[CODE]{encoded_url}[/CODE]"
else:
    text_to_hide = f"[DOWNCLOUD]{Link}[/DOWNCLOUD]"


clear_output()
##################### Post Prep ############################

%cd "/content"
clear_output()
print(locals()[template_to_use](
    text_to_hide, 
    imdb_data, 
    YouTube_URL,
    source_string, 
    media_info, 
    SSURLs, 
    min_posts, 
    min_reaction_score
    ))

####################### Trying to Guess Title String #######################
if generate_title:
    print("\n\n", generate_title_mediainfo(Path, get_media_info_dict(Path), pass_thru))